In [2]:
import numpy as np
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Hyderabad').text

In [5]:
pd.set_option('max_colwidth', 800) #for better view
# Get the text data od neighbours in mumbai from wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Hyderabad').text 
soup = BeautifulSoup(source, 'lxml')
print(soup.find('li'))

<li class="toclevel-1 tocsection-1"><a href="#Central"><span class="tocnumber">1</span> <span class="toctext">Central</span></a>
<ul>
<li class="toclevel-2 tocsection-2"><a href="#Ameerpet"><span class="tocnumber">1.1</span> <span class="toctext">Ameerpet</span></a></li>
<li class="toclevel-2 tocsection-3"><a href="#Sanathnagar"><span class="tocnumber">1.2</span> <span class="toctext">Sanathnagar</span></a></li>
<li class="toclevel-2 tocsection-4"><a href="#Khairatabad"><span class="tocnumber">1.3</span> <span class="toctext">Khairatabad</span></a></li>
<li class="toclevel-2 tocsection-5"><a href="#Musheerabad"><span class="tocnumber">1.4</span> <span class="toctext">Musheerabad</span></a></li>
<li class="toclevel-2 tocsection-6"><a href="#Amberpet"><span class="tocnumber">1.5</span> <span class="toctext">Amberpet</span></a></li>
<li class="toclevel-2 tocsection-7"><a href="#Nampally"><span class="tocnumber">1.6</span> <span class="toctext">Nampally</span></a></li>
<li class="toclevel-

In [6]:
table_post = soup.find('li')
mumareas = table_post.find_all('span')
neighbourhood = []

for i in range(0, len(mumareas),2):
    neighbourhood.append(mumareas[i+1].text.strip())
    
        
df_Hyderabad= pd.DataFrame(data=[neighbourhood]).transpose()
df_Hyderabad.columns = ['Neighbourhood']
print(df_Hyderabad)

         Neighbourhood
0              Central
1             Ameerpet
2          Sanathnagar
3          Khairatabad
4          Musheerabad
5             Amberpet
6             Nampally
7         Secunderabad
8  Secunderabad Cantt.


In [7]:
df_Hyderabad.shape

(9, 1)

### Cleaning 

In [25]:
print(df_Hyderabad)

  Neighbourhood
0       Central
1      Ameerpet
2   Sanathnagar
3   Khairatabad
4   Musheerabad
5      Amberpet
6      Nampally
7  Secunderabad


In [22]:
#to find hyderabad coordinates
address = 'Hyderabad, India'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad City are 17.38878595, 78.46106473453146.


In [27]:
#to find each area coordinates
from geopy.geocoders import Nominatim
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array
for area in df_Hyderabad["Neighbourhood"] : 
    place_name = area + ",Hyderabad, India"
    geolocator = Nominatim(user_agent="googleapis")
    location = geolocator.geocode(area)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)

In [28]:
df_Hyderabad['Latitude'] = latitudes #append the data to dataframe
df_Hyderabad['Longitude'] = longitudes

In [29]:
df_Hyderabad.head()

,Neighbourhood,Latitude,Longitude
0,Central,9.577626,-68.424820
1,Ameerpet,17.437501,78.448251
2,Sanathnagar,17.456965,78.443478
3,Khairatabad,17.411771,78.462200
4,Musheerabad,17.419142,78.498573


In [30]:
hyd_lat = 17.38878595
hyd_lng = 78.46106473453146 

# Creates map of hyd using latitude and longitude values
map_hyd = folium.Map(location=[hyd_lat, hyd_lng], zoom_start=10)

# Add markers to map
for lat, lng, neighbourhood in zip(df_Hyderabad['Latitude'], df_Hyderabad['Longitude'], df_Hyderabad['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hyd)  
    
map_hyd

In [31]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
# The code was removed by Watson Studio for sharing.

In [33]:
explore_df_list = []

for i, nbd_name in enumerate(df_Hyderabad['Neighbourhood']):  
    
    try :
        ### Getting the data of neighbourhood
        nbd_name = df_Hyderabad.loc[i, 'Neighbourhood']
        nbd_lat = df_Hyderabad.loc[i, 'Latitude']
        nbd_lng = df_Hyderabad.loc[i, 'Longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 30 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    
        
        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

In [34]:
df_venue = pd.DataFrame([item for item in explore_df_list])
df_venue.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
df_venue.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Ameerpet,17.437501,78.448251,Kakatiya Deluxe Mess,Diner,17.433435,78.447090
1,Ameerpet,17.437501,78.448251,Mekong,Chinese Restaurant,17.437151,78.454301
2,Ameerpet,17.437501,78.448251,Blue Fox,Indian Restaurant,17.437054,78.445912
3,Ameerpet,17.437501,78.448251,10 Downing Street,Pub,17.435868,78.457443
4,Ameerpet,17.437501,78.448251,Hotel Green Park,Hotel,17.436276,78.453892


In [35]:
# One hot encoding
hyd_onehot = pd.get_dummies(df_venue[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
hyd_onehot['Neighbourhood'] = df_venue['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [hyd_onehot.columns[-1]] + hyd_onehot.columns[:-1].values.tolist()
hyd_onehot = hyd_onehot[fixed_columns]

hyd_onehot.head()

,Neighbourhood,Bakery,Bank,Bar,Basketball Court,Bowling Alley,Bus Station,Café,Chinese Restaurant,Clothing Store,...,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Shoe Store,South Indian Restaurant,Spa,Train Station,Vegetarian / Vegan Restaurant
0,Ameerpet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ameerpet,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Ameerpet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ameerpet,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Ameerpet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
hyd_grouped = hyd_onehot.groupby('Neighbourhood').mean().reset_index()
hyd_grouped.head()

,Neighbourhood,Bakery,Bank,Bar,Basketball Court,Bowling Alley,Bus Station,Café,Chinese Restaurant,Clothing Store,...,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Shoe Store,South Indian Restaurant,Spa,Train Station,Vegetarian / Vegan Restaurant
0,Ameerpet,0.033333,0.000000,0.033333,0.000,0.000,0.000,0.100,0.066667,0.033333,...,0.000000,0.033333,0.000,0.066667,0.000000,0.000,0.000000,0.000,0.000,0.033333
1,Khairatabad,0.000000,0.035714,0.000000,0.000,0.000,0.000,0.000,0.000000,0.000000,...,0.035714,0.000000,0.000,0.035714,0.035714,0.000,0.035714,0.000,0.000,0.071429
2,Musheerabad,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000000
3,Sanathnagar,0.000000,0.000000,0.000000,0.125,0.000,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000,0.125,0.000000
4,Secunderabad,0.000000,0.000000,0.000000,0.000,0.125,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.125,0.000000,0.000000,0.125,0.000000,0.125,0.000,0.000000


In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = hyd_grouped['Neighbourhood']

for ind in np.arange(hyd_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hyd_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ameerpet,Indian Restaurant,Café,Hotel,Chinese Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Bar,Clothing Store,Department Store
1,Khairatabad,Multiplex,Hotel,Coffee Shop,Indian Restaurant,Vegetarian / Vegan Restaurant,Scenic Lookout,Movie Theater,Fast Food Restaurant,Garden,Sandwich Place
2,Musheerabad,Coffee Shop,Convenience Store,Grocery Store,Ice Cream Shop,Garden,Fast Food Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega
3,Sanathnagar,Park,Deli / Bodega,Train Station,Basketball Court,Bus Station,Café,Department Store,Motorcycle Shop,Garden,Fast Food Restaurant
4,Secunderabad,Ice Cream Shop,Spa,Hookah Bar,Shoe Store,Restaurant,Bowling Alley,Men's Store,Vegetarian / Vegan Restaurant,Convenience Store,Diner


In [40]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

hyd_grouped_clustering = hyd_grouped.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 2, 1, 0], dtype=int32)

In [41]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighbourhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Ameerpet,Indian Restaurant,Café,Hotel,Chinese Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Bar,Clothing Store,Department Store
1,4,Khairatabad,Multiplex,Hotel,Coffee Shop,Indian Restaurant,Vegetarian / Vegan Restaurant,Scenic Lookout,Movie Theater,Fast Food Restaurant,Garden,Sandwich Place
2,2,Musheerabad,Coffee Shop,Convenience Store,Grocery Store,Ice Cream Shop,Garden,Fast Food Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega
3,1,Sanathnagar,Park,Deli / Bodega,Train Station,Basketball Court,Bus Station,Café,Department Store,Motorcycle Shop,Garden,Fast Food Restaurant
4,0,Secunderabad,Ice Cream Shop,Spa,Hookah Bar,Shoe Store,Restaurant,Bowling Alley,Men's Store,Vegetarian / Vegan Restaurant,Convenience Store,Diner


In [42]:
hyd_merged = df_Hyderabad

hyd_merged = hyd_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

hyd_merged.head()

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central,9.577626,-68.424820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ameerpet,17.437501,78.448251,3.0,Indian Restaurant,Café,Hotel,Chinese Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Bar,Clothing Store,Department Store
2,Sanathnagar,17.456965,78.443478,1.0,Park,Deli / Bodega,Train Station,Basketball Court,Bus Station,Café,Department Store,Motorcycle Shop,Garden,Fast Food Restaurant
3,Khairatabad,17.411771,78.462200,4.0,Multiplex,Hotel,Coffee Shop,Indian Restaurant,Vegetarian / Vegan Restaurant,Scenic Lookout,Movie Theater,Fast Food Restaurant,Garden,Sandwich Place
4,Musheerabad,17.419142,78.498573,2.0,Coffee Shop,Convenience Store,Grocery Store,Ice Cream Shop,Garden,Fast Food Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega


In [43]:
hyd_merged=hyd_merged.dropna()

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighbourhood'], hyd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analysis

In [45]:

#for cluster 1
i=1
hyd_merged.loc[hyd_merged['Cluster Labels'] == (i - 1), hyd_merged.columns[[0] + np.arange(4, hyd_merged.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Secunderabad,Ice Cream Shop,Spa,Hookah Bar,Shoe Store,Restaurant,Bowling Alley,Men's Store,Vegetarian / Vegan Restaurant,Convenience Store,Diner


In [47]:

#for cluster 2
i=2
hyd_merged.loc[hyd_merged['Cluster Labels'] == (i - 1), hyd_merged.columns[[0] + np.arange(4, hyd_merged.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sanathnagar,Park,Deli / Bodega,Train Station,Basketball Court,Bus Station,Café,Department Store,Motorcycle Shop,Garden,Fast Food Restaurant


In [48]:

#for cluster 3
i=3
hyd_merged.loc[hyd_merged['Cluster Labels'] == (i - 1), hyd_merged.columns[[0] + np.arange(4, hyd_merged.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Musheerabad,Coffee Shop,Convenience Store,Grocery Store,Ice Cream Shop,Garden,Fast Food Restaurant,Diner,Dessert Shop,Department Store,Deli / Bodega


#### sanathnagar area has fast food centre as tenth most common place. so we can suggest owner to open a new fast food centre in that area.